In [1]:
#data prefix for output data
yearmonth = "202405"

In [2]:
#import necessary packages
import platform
import os
import sys
import time
#for managing the data
import pandas as pd

#install !pip install webdriver-manager
#webdrivers
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By

from bs4 import BeautifulSoup

import pathlib

import os
from io import StringIO

from tqdm import tqdm

In [3]:
#get current path
curr_path = os.getcwd()
project_tag = curr_path.split("/")[-1]
grouping = curr_path.split("/")[-2]
output_path = "../../../data/{}/{}/{}/".format(grouping,project_tag,yearmonth )

#create data output path if it doesn't exist
pathlib.Path(output_path).mkdir(parents=True, exist_ok=True) 

In [4]:
#load in level 1 data
df = pd.read_csv(output_path + "1_level_table.csv")
print(df.shape)

(1340, 9)


In [5]:
#open web driver
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))

In [6]:
#place holder for final dataframe list
df_list = []

for i,r in tqdm( df.iterrows() ):
    url = r['url_entity_partial']
    url_entity = r['url_entity_full']
    
    #load the main landing page
    driver.get(url_entity)
    
    try:
        #find disclaimer button
        disclamer_btn = driver.find_element(by='name', value="AcceptDisclaimerButton")
    
        #click the disclaimer button to move to the data page
        disclamer_btn.click()
    except:
        pass

    #place holder for extracted data
    data_url_list = []
    data_url_full_list = []
    
    #locate each table rows
    data_rows = driver.find_elements(By.CLASS_NAME, value="hovereffect")

    #if no inspection data skip to the next url
    if len(data_rows)==0:
        continue
    
    for row in data_rows:
        data_url_list.append( row.get_attribute("onclick") )
    
    url_prefix = "http://choosewise.nwhu.on.ca"
    for u in data_url_list:
        temp_url = url_prefix+u.split("'")[1]
        data_url_full_list.append(temp_url)
    
    tables = pd.read_html(StringIO((driver.page_source)))
    table  = tables[1]
    table['url_inspection'] = data_url_list
    table['url_inspection_full'] = data_url_full_list
    table['url_entity_full'] = url_entity

    df_list.append(table)
 

1340it [08:20,  2.68it/s]


In [7]:
final_df = pd.concat(df_list)
print(final_df.shape)

(1614, 8)


In [8]:
final_df

Inspection Date Inspection Type  Critical Infractions Count  \
0      13-Dec-2023        Required                           0   
1      22-Jun-2023        Required                           1   
0      15-Feb-2024       Follow-up                           0   
1      09-Feb-2024       Follow-up                           0   
2      05-Feb-2024        Required                           0   
..             ...             ...                         ...   
0      15-Jun-2023        Required                           0   
1      07-Sep-2022        Required                           0   
2      23-Jul-2021        Required                           0   
0      29-Jan-2020        Required                           0   
0      30-Jan-2020        Required                           0   

    Non Critical Infractions Count  Inspection Result  \
0                                0                NaN   
1                                0                NaN   
0                                0                NaN   
1                                1                NaN   
2                                2                NaN   
..                             ...                ...   
0                                0                NaN   
1                                0                NaN   
2                                0                NaN   
0                                0                NaN   
0                                0                NaN   

                                       url_inspection  \
0   location.href = '/Inspection/Details/ed140389-...   
1   location.href = '/Inspection/Details/33b531bf-...   
0   location.href = '/Inspection/Details/c34617a8-...   
1   location.href = '/Inspection/Details/52572a34-...   
2   location.href = '/Inspection/Details/e671481f-...   
..                                                ...   
0   location.href = '/Inspection/Details/f4a145ad-...   
1   location.href = '/Inspection/Details/584a722e-...   
2   location.href = '/Inspection/Details/b4b4ad9a-...   
0   location.href = '/Inspection/Details/466f9154-...   
0   location.href = '/Inspection/Details/70a1e28a-...   

                                  url_inspection_full  \
0   http://choosewise.nwhu.on.ca/Inspection/Detail...   
1   http://choosewise.nwhu.on.ca/Inspection/Detail...   
0   http://choosewise.nwhu.on.ca/Inspection/Detail...   
1   http://choosewise.nwhu.on.ca/Inspection/Detail...   
2   http://choosewise.nwhu.on.ca/Inspection/Detail...   
..                                                ...   
0   http://choosewise.nwhu.on.ca/Inspection/Detail...   
1   http://choosewise.nwhu.on.ca/Inspection/Detail...   
2   http://choosewise.nwhu.on.ca/Inspection/Detail...   
0   http://choosewise.nwhu.on.ca/Inspection/Detail...   
0   http://choosewise.nwhu.on.ca/Inspection/Detail...   

                                      url_entity_full  
0   http://choosewise.nwhu.on.ca/Facility/Details/...  
1   http://choosewise.nwhu.on.ca/Facility/Details/...  
0   http://choosewise.nwhu.on.ca/Facility/Details/...  
1   http://choosewise.nwhu.on.ca/Facility/Details/...  
2   http://choosewise.nwhu.on.ca/Facility/Details/...  
..                                                ...  
0   http://choosewise.nwhu.on.ca/Facility/Details/...  
1   http://choosewise.nwhu.on.ca/Facility/Details/...  
2   http://choosewise.nwhu.on.ca/Facility/Details/...  
0   http://choosewise.nwhu.on.ca/Facility/Details/...  
0   http://choosewise.nwhu.on.ca/Facility/Details/...  

[1614 rows x 8 columns]

In [9]:
final_df.to_csv( output_path + "2_level_table.csv", index=False)

In [10]:
#close webdriver
try:
    driver.close()
except:
    pass